# Resnext101

In [1]:
%matplotlib inline

In [2]:
import pretrainedmodels

from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

from sklearn import metrics

In [3]:
PATH='../data/raw/HE_DL/'
arch=resnext101
sz=350; bs=32

In [4]:
label_csv=f'{PATH}meta-data/train.csv'
n=len(list(open(label_csv))) - 1
val_idxs=get_cv_idxs(n)
n, len(val_idxs)

(13000, 2600)

## Initial Model

In [5]:
def get_data(sz, bs, val_idxs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.05)
    data = ImageClassifierData.from_csv(PATH, 
                                        'train/', 
                                        f'{PATH}meta-data/train.csv',
                                        bs=bs,
                                        tfms=tfms,
                                        val_idxs=val_idxs, # to make sure only single image is there in validation set
                                        test_name='test/'
                                       )
    
    return data if sz > 300 else data.resize(750, 'tmp/')

## Full Training

In [18]:
val_idxs = [0]

data  = get_data(sz, bs, val_idxs)
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 1/1 [00:00<00:00, 19.82it/s]


In [19]:
learn.fit(1e-2, 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.232149   0.000218   1.0       
    1      0.160243   6.2e-05    1.0                         



[array([ 0.00006]), 1.0]

In [20]:
learn.precompute=False
learn.fit(1e-2, 5, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.169561   9.2e-05    1.0       
    1      0.12242    2.2e-05    1.0                         
    2      0.153519   1.7e-05    1.0                         
    3      0.098577   1.1e-05    1.0                          
    4      0.115265   4.2e-05    1.0                          



[array([ 0.00004]), 1.0]

In [21]:
learn.set_data(get_data(450, bs=32,val_idxs=val_idxs))
learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.09427    2.7e-05    1.0       
    1      0.095833   4e-06      1.0                          
    2      0.089385   3.2e-05    1.0                          



[array([ 0.00003]), 1.0]

In [22]:
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.097685   4e-06      1.0       
    1      0.08134    6e-06      1.0                          
    2      0.073029   2e-06      1.0                          
    3      0.091427   4e-06      1.0                          
    4      0.071005   0.0        1.0                          
    5      0.062873   2e-06      1.0                          
    6      0.061572   2e-06      1.0                          



[array([ 0.]), 1.0]

## Create Submission

In [23]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs        = np.mean(np.exp(log_preds), 0)

In [24]:
df         = pd.DataFrame(probs)
df.columns = data.classes

df.insert(0, 'image_id', [o.split('/')[1] for o in data.test_ds.fnames])
df.loc[:, 'img_num'] = [int(f.split('-')[1].split('.')[0]) for f in data.test_ds.fnames]

df = df.sort_values(by='img_num')
df.drop('img_num', axis=1, inplace=True)

df.to_csv('../data/raw/HE_DL/submissions/sub18.csv', index=False)